First, load the data,in this notebook we are using the smokoy mountain data as the example. And will use basic AE to get the transformation of the data and do prediction using the same models.

In [1]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

2025-01-17 00:46:31.137869: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 00:46:31.140285: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 00:46:31.143834: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 00:46:31.153944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737092791.170733 1571361 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737092791.17

In [2]:
import pandas as pd
import numpy as np
import random
from sklearn import svm
import pickle
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available:  0


2025-01-17 00:46:33.685371: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### 1. Load the data and data preprocess
After reading the data, convert the abundance matrix into relative abundance matrix; for the response variables, since we have multiple variables, organize them into an array with each variable as a column.

In [4]:
data,burn_label,un_label,duration_label,ASVs,df=loadData.loadSoilData("../data/count_table/merge_proportion_asv_smoky_moutain.csv")

In [5]:
data = FS.relative_abundance(data)

In [50]:
from tensorflow.keras import layers
from tensorflow import keras as ks
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras.models import Model

# --------------------------------------------------------------------------------------------------#

# To define your model, use the Keras Model Subclassing API.
class Autoencoder(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape).numpy(), activation='relu'),
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded



### 2.AE


In [6]:
data_train, data_test, y_train, y_test = RunML_continue.split_and_scale_data(data,burn_label)
print(data_train.shape)
print(data_test.shape)

(41, 1768)
(18, 1768)


In [26]:
data_train.shape[1]

1768

In [51]:
latent_dim = 20
shape = data_train.shape[1:]

autoencoder = Autoencoder(latent_dim, shape)
autoencoder.compile(optimizer='adam', loss=ks.losses.MeanSquaredError())

In [52]:
autoencoder.fit(data_train, data_train,
                epochs=10,
                shuffle=True,
                validation_data=(data_test, data_test))

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - loss: 0.1967 - val_loss: 1.5810
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1982 - val_loss: 1.5784
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1975 - val_loss: 1.5761
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.1889 - val_loss: 1.5741
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.1944 - val_loss: 1.5724
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1978 - val_loss: 1.5709
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1969 - val_loss: 1.5697
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.1929 - val_loss: 1.5686
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1977 - val_loss: 1.5675
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1972 - val_loss: 1.5663


In [53]:
data_AE = autoencoder.encoder(data).numpy()

In [54]:
iter =30
cls = ["RF","SVM", "CatBoost","NB"]

data_subset = {"AllFeatures":data, 
               "AE": data_AE
              }

In [56]:
print(np.shape(data))
print(np.shape(data_AE))

(59, 1768)
(59, 20)


The  function will print out the accuracy and AUC for each dataset using each classifier, and also will return the y_actual, y_predict, y_predprob for future use.

In [57]:
targetLabel=burn_label
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.780303    0.922917    0.746970    0.843750      0.845455        
AE            0.710606    0.730903    0.777273    0.821528      0.589394        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.887037     0.863636    0.801389  
AE             0.630556     0.743939    0.722917  


In [58]:
print(metric.metric_sum(dict_cm))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.779661  1.000000   0.235294    1.000000     0.423886
AllFeatures_SVM       0.745763  0.571429   0.470588    0.857143     0.348896
AllFeatures_CatBoost  0.847458  1.000000   0.470588    1.000000     0.622530
AllFeatures_NB        0.864407  0.846154   0.647059    0.952381     0.655004
AE_RF                 0.711864  0.500000   0.529412    0.785714     0.309960
AE_SVM                0.779661  0.625000   0.588235    0.857143     0.453716
AE_CatBoost           0.593220  0.347826   0.470588    0.642857     0.105347
AE_NB                 0.745763  0.555556   0.588235    0.809524     0.391238


In [59]:
targetLabel=un_label
dict_cm_un = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.556061    0.702698    0.556061    0.424444      0.524242        
AE            0.524242    0.545556    0.557576    0.508254      0.507576        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.582540     0.610606    0.605000  
AE             0.514286     0.509091    0.448889  


In [61]:
print(metric.metric_sum(dict_cm_un))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.559322  0.500000   0.230769    0.818182     0.060376
AllFeatures_SVM       0.559322  0.500000   0.346154    0.727273     0.079171
AllFeatures_CatBoost  0.525424  0.400000   0.153846    0.818182    -0.037014
AllFeatures_NB        0.610169  0.551724   0.615385    0.606061     0.219913
AE_RF                 0.525424  0.458333   0.423077    0.606061     0.029448
AE_SVM                0.559322  0.500000   0.461538    0.636364     0.098945
AE_CatBoost           0.508475  0.440000   0.423077    0.575758    -0.001171
AE_NB                 0.508475  0.454545   0.576923    0.454545     0.031469


In [60]:
targetLabel=duration_label
dict_cm_dur = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.590909    0.457857    0.507576    0.471607      0.542424        
AE            0.560606    0.455804    0.506061    0.365179      0.575758        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.436429     0.507576    0.516071  
AE             0.569107     0.554545    0.597589  


In [62]:
print(metric.metric_sum(dict_cm_dur))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.593220  0.333333   0.090909    0.891892    -0.027517
AllFeatures_SVM       0.508475  0.230769   0.136364    0.729730    -0.156230
AllFeatures_CatBoost  0.542373  0.222222   0.090909    0.810811    -0.132182
AllFeatures_NB        0.508475  0.360000   0.409091    0.567568    -0.022842
AE_RF                 0.559322  0.357143   0.227273    0.756757    -0.018154
AE_SVM                0.508475  0.294118   0.227273    0.675676    -0.103625
AE_CatBoost           0.576271  0.384615   0.227273    0.783784     0.012900
AE_NB                 0.559322  0.437500   0.636364    0.513514     0.145476


In [49]:
shape = data_train.shape[1:]
print(shape)

(1768,)


In [68]:
param_grid = {
            #'input_dim': [X_train_scaled.shape[1]], 
            'latent_dim': [10, 25, 50, 100],
            'activation': ['relu', 'sigmoid', 'tanh']  
        }


In [71]:
best_score = float('inf')

for activation in param_grid['activation']:
    class Autoencoder(Model):
      def __init__(self, latent_dim, shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.shape = shape
        self.encoder = tf.keras.Sequential([
          layers.Flatten(),
          layers.Dense(latent_dim, activation=activation),
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(tf.math.reduce_prod(shape).numpy(), activation=activation),
          layers.Reshape(shape)
        ])
    
      def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    for latent_dim in param_grid['latent_dim']:
        autoencoder = Autoencoder(latent_dim, shape)
        autoencoder.compile(optimizer='adam', loss=ks.losses.MeanSquaredError())
        autoencoder.fit(data_train, data_train,
                    epochs=10,
                    shuffle=True,
                    verbose=0,
                    validation_data=(data_test, data_test))
        val_loss = autoencoder.evaluate(data_test, data_test, verbose=0)
        if val_loss < best_score:
            best_score = val_loss
            print(f"activation:{activation}, latent_dim:{latent_dim}, val_loss:{val_loss}")
    
        
        
            

activation:relu, latent_dim:10, val_loss:1.5651246309280396
activation:relu, latent_dim:100, val_loss:1.5317176580429077


In [ ]:
## latent_dim = [10, 25, 50, 100]
for dim in latent_dim:
    print(f"latent dim: {dim}")
    autoencoder = Autoencoder(dim, shape)
    autoencoder.compile(optimizer='adam', loss=ks.losses.MeanSquaredError())
    
    autoencoder.fit(data_train, data_train,
                    epochs=10,
                    shuffle=True,
                    validation_data=(data_test, data_test))
    
    data_AE = autoencoder.encoder(data).numpy()



In [74]:
class Autoencoder(Model):
      def __init__(self, latent_dim, shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.shape = shape
        self.encoder = tf.keras.Sequential([
          layers.Flatten(),
          layers.Dense(latent_dim, activation='relu'),
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(tf.math.reduce_prod(shape).numpy(), activation='relu'),
          layers.Reshape(shape)
        ])
    
      def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    
autoencoder = Autoencoder(100, shape)
autoencoder.compile(optimizer='adam', loss=ks.losses.MeanSquaredError())
autoencoder.fit(data_train, data_train,
            epochs=10,
            shuffle=True,
            verbose=0,
            validation_data=(data_test, data_test))
 

In [75]:
data_AE = autoencoder.encoder(data).numpy()

In [76]:
data_subset = {"AllFeatures":data, 
               "AE": data_AE
              }

In [77]:
targetLabel=burn_label
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.780303    0.922917    0.746970    0.843750      0.845455        
AE            0.763636    0.759606    0.830303    0.809259      0.775758        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.887037     0.863636    0.801389  
AE             0.849769     0.760606    0.750347  


In [81]:
print(metric.metric_sum(dict_cm))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.779661  1.000000   0.235294    1.000000     0.423886
AllFeatures_SVM       0.745763  0.571429   0.470588    0.857143     0.348896
AllFeatures_CatBoost  0.847458  1.000000   0.470588    1.000000     0.622530
AllFeatures_NB        0.864407  0.846154   0.647059    0.952381     0.655004
AE_RF                 0.762712  0.666667   0.352941    0.928571     0.354602
AE_SVM                0.830508  0.769231   0.588235    0.928571     0.564712
AE_CatBoost           0.779661  0.600000   0.705882    0.809524     0.493119
AE_NB                 0.762712  0.588235   0.588235    0.833333     0.421569


In [79]:
targetLabel=duration_label
dict_cm_dur = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.590909    0.457857    0.507576    0.471607      0.542424        
AE            0.592424    0.478304    0.507576    0.444107      0.407576        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.436429     0.507576    0.516071  
AE             0.417143     0.543939    0.527232  


In [80]:
print(metric.metric_sum(dict_cm_dur))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.593220  0.333333   0.090909    0.891892    -0.027517
AllFeatures_SVM       0.508475  0.230769   0.136364    0.729730    -0.156230
AllFeatures_CatBoost  0.542373  0.222222   0.090909    0.810811    -0.132182
AllFeatures_NB        0.508475  0.360000   0.409091    0.567568    -0.022842
AE_RF                 0.593220  0.437500   0.318182    0.756757     0.081512
AE_SVM                0.508475  0.181818   0.090909    0.756757    -0.189144
AE_CatBoost           0.406780  0.117647   0.090909    0.594595    -0.335798
AE_NB                 0.542373  0.400000   0.454545    0.594595     0.048088


In [82]:
targetLabel=un_label
dict_cm_un = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.556061    0.702698    0.556061    0.424444      0.524242        
AE            0.574242    0.649286    0.590909    0.595397      0.456061        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.582540     0.610606    0.605000  
AE             0.598889     0.539394    0.585714  


In [83]:
print(metric.metric_sum(dict_cm_un))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.559322  0.500000   0.230769    0.818182     0.060376
AllFeatures_SVM       0.559322  0.500000   0.346154    0.727273     0.079171
AllFeatures_CatBoost  0.525424  0.400000   0.153846    0.818182    -0.037014
AllFeatures_NB        0.610169  0.551724   0.615385    0.606061     0.219913
AE_RF                 0.576271  0.523810   0.423077    0.696970     0.124478
AE_SVM                0.593220  0.541667   0.500000    0.666667     0.168443
AE_CatBoost           0.457627  0.350000   0.269231    0.606061    -0.130796
AE_NB                 0.542373  0.483871   0.576923    0.515152     0.091543
